In [16]:
import pandas as pd
import numpy as np
import functools as ft
from math import radians, cos, sin, asin, sqrt

### Paper data (given in the papers_processed folder)

In [17]:
df_papers = [pd.read_excel('papers_processed/papers_final.xlsx')]
df_papers.append(pd.read_excel('papers_processed/gabriel.xlsx'))
df_papers.append(pd.read_excel('papers_processed/karmanya_main_results_new.xlsx'))
df_papers.append(pd.read_excel('papers_processed/maik.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper16_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper18_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper21_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper29_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper30_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper33_data.xlsx'))

### Final grid data with yield (given in the combined data folder)

In [18]:
df_data = pd.read_csv('combined data/soilgrids_agri4cast_yield.csv')

### Filtering data based on distance to its cloasest yield location

In [19]:
df_data = df_data[df_data['nearest_distance_km'] < 40]

### Changing the range of soil ph to 0-14 (0-140 befors)

In [20]:
df_data['phh2o_0-5cm_mean'], df_data['phh2o_5-15cm_mean'],df_data['phh2o_15-30cm_mean'],df_data['phh2o_30-60cm_mean'],df_data['phh2o_60-100cm_mean'],df_data['phh2o_100-200cm_mean'] = df_data['phh2o_0-5cm_mean']/10, df_data['phh2o_5-15cm_mean']/10,df_data['phh2o_15-30cm_mean']/10,df_data['phh2o_30-60cm_mean']/10,df_data['phh2o_60-100cm_mean']/10,df_data['phh2o_100-200cm_mean']/10

### Code responsible for processing numerical selection values (pH etc.)

In [21]:
def process_numerical(env_data, row, selection_env, temp):
    selection = row['selection']
    if "-" in selection:
        bounds = selection.split('-')
        
        satisfying_rows = env_data[(env_data[selection_env] > float(bounds[0]))]
        satisfying_rows = satisfying_rows[(satisfying_rows[selection_env] < float(bounds[1]))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        
    elif ">" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] > float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
    elif "<" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] < float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
    return temp

### This is just a map for soilGrids, which maps soil type names to ids

In [22]:
soil_type_map = {
    'Acrisols': 0,
    'Albeluvisols': 1,
    'Alisols': 2,
    'Andosols': 3,
    'Arenosols': 4,
    'Calcisols': 5,
    'Cambisols': 6,
    'Chernozems': 7,
    'Cryosols': 8,
    'Durisols': 9,
    'Ferralsols': 10,
    'Fluvisols': 11,
    'Gleysols': 12,
    'Gypsisols': 13,
    'Histosols': 14,
    'Kastanozems': 15,
    'Leptosols': 16,
    'Lixisols': 17,
    'Luvisols': 18,
    'Nitisols': 19,
    'Phaeozems': 20,
    'Planosols': 21,
    'Plinthosols': 22,
    'Podzols': 23,
    'Regosols': 24,
    'Solonchaks': 25,
    'Solonetz': 26,
    'Stagnosols': 27,
    'Umbrisols': 28,
    'Vertisols': 29
}

### Code responsible for processing soil type selection, either by clay, sand and silt content or scientific name

In [23]:
def filter_by_soil_type(env_data, row, temp):
    soil_type = row['selection']
    
    # Convert g/kg to percentage
    sand_pct = env_data['sand_0-5cm_mean'] / 10
    silt_pct = env_data['silt_0-5cm_mean'] / 10
    clay_pct = env_data['clay_0-5cm_mean'] / 10
    ph = env_data['phh2o_0-5cm_mean']
    condition = "THIS IS A WALLUE"
    # Define conditions for each soil type
    if soil_type.lower() == 'sandy':
        
        condition = (sand_pct > 85) 
    
    elif soil_type.lower() == 'loamy':
        condition = (
            (sand_pct <= 52) & 
            (silt_pct >= 28) & (silt_pct <= 50) & 
            (clay_pct >= 7) & (clay_pct <= 27)
        )
    
    elif soil_type.lower() == 'clay':
        condition = (sand_pct <= 45) & (silt_pct <= 40) & (clay_pct >= 40)
    elif soil_type.lower() == 'sandy,acidic':
        condition = (sand_pct > 85) & (silt_pct + 1.5 * clay_pct < 15) & (ph <7)
    elif soil_type.lower() == 'lixisols':
        satisfying_rows = env_data[env_data['MostProbable'] == soil_type_map['Lixisols']]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        return temp
    elif soil_type.lower() == 'arenosols':
        satisfying_rows = env_data[env_data['MostProbable'] == soil_type_map['Arenosols']]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        return temp
    elif soil_type.lower() == 'acrisols':
        satisfying_rows = env_data[env_data['MostProbable'] == soil_type_map['Acrisols']]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        return temp
    else:
        print(
            f"Invalid soil_type '{soil_type}'. "
            "Must be 'sandy', 'loamy', or 'clay'"
        )
        return temp
    satisfying_rows = env_data[condition]
    satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
    temp.append(satisfying_rows)
    return temp

### Code responsible for processing region selections, by creating lat-lon bounding boxes on the region

In [24]:
def filter_by_region(env_data, row, temp):
    region = row['selection']
    
    
    lon = env_data['lon'] 
    lat = env_data['lat'] 
    
    condition = "THIS IS A WALLUE"
    
    if region.lower() == 'spain':
        condition = (lon >= -9.30) & (lon <= 3.33) & (lat >= -36.00) & (lat <= 43.80)
    
    elif region.lower() == 'turkey':
        condition = (lon >= 25.00) & (lon <= 45.00) & (lat >= 35.8) & (lat <= 42.1)
    else:
        print(
            f"Invalid region '{region}'. "
            "Must be 'spain', or 'turkey'"
        )
        return temp
    satisfying_rows = env_data[condition]
    satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
    temp.append(satisfying_rows)
    return temp

### Map between selection names in our paper Excel data and how they are called in the grid data

In [25]:
process_dict = {
    'yearly precipitation': 'yearly_rain_mean',
    'latitude': 'lat',
    'soil ph': 'phh2o_0-5cm_mean',
    'yearly temperature': 'yearly_avg_mean_temp_mean'
}

### This code calls the previous 3 functions to do the actual selection of grids

In [26]:
def combine(env_data, paper_data):
    temp = []
    for idx, row in paper_data.iterrows():
        sel_type = row['selection_type'].lower()
        if sel_type in process_dict.keys():
            temp = process_numerical(env_data, row, process_dict[sel_type], temp)
        elif sel_type == "soil_type":
            temp = filter_by_soil_type(env_data, row, temp)
        elif sel_type == "region":
            temp = filter_by_region(env_data, row, temp)
    combined = pd.DataFrame(columns=['results_detailed','farming_practice','results_actual'])   
    if len(temp) > 0:
        combined = pd.concat(temp, ignore_index=True)   
    return combined
            
                

##### Simple renaming

In [27]:
df_papers[4]['selection_type'] = df_papers[4]['selection type']

### Calling the combine function for each paper and than concatenating the grids into a single one

In [28]:
combined = []
for idx, data in enumerate(df_papers):
    combined.append(combine(df_data, data))
combined_fin = pd.concat(combined, ignore_index=True)  
combined_fin

Invalid region '
Turkey'. Must be 'spain', or 'turkey'


C:\Users\artur\AppData\Local\Temp\ipykernel_18968\298452190.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
C:\Users\artur\AppData\Local\Temp\ipykernel_18968\298452190.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practic

,Unnamed: 0,lon,lat,yearly_rain_6,yearly_rain_7,yearly_rain_8,yearly_rain_9,yearly_rain_10,yearly_rain_11,yearly_rain_12,...,cec_100-200cm_mean,MostProbable,yield,harvest_area,production,adm_id,nearest_distance_km,results_detailed,results_actual,farming_practice
0,2168,25.268266,34.802836,325.10010,266.00003,233.80005,371.40000,367.30010,465.50012,504.89987,...,205.71428,18.0,1.470227,275.213636,403.251455,EL431,39.872395,yield,"""+25""",biochar amendment (lat <35)
1,2251,23.997782,35.120457,533.10000,353.30014,335.10004,541.00000,373.70007,479.60013,613.00020,...,185.82806,18.0,1.828154,10.953846,18.148538,EL434,28.912033,yield,"""+15.05""",SSDI vs SDI (ph <7)
2,2256,25.585887,35.120457,302.60007,255.69998,233.40012,360.70013,370.49997,495.10010,521.20000,...,194.68938,18.0,2.304045,248.818136,516.362818,EL432,22.766928,yield,"""+15.05""",SSDI vs SDI (ph <7)
3,2333,23.997782,35.438078,569.10004,361.60000,347.80002,563.90010,378.70007,505.30020,636.80000,...,203.05536,18.0,1.828154,10.953846,18.148538,EL434,10.015525,yield,"""+15.05""",SSDI vs SDI (ph <7)
4,2514,-5.540971,36.390941,532.40010,431.30014,702.59990,755.80020,1229.39950,566.89984,525.60000,...,168.48402,18.0,2.814870,12440.652174,36517.956522,ES612,26.769221,yield,"""+15.05""",SSDI vs SDI (ph <7)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33588,16479,26.221129,64.976831,423.20004,539.60000,530.90000,405.50012,437.20010,618.69990,587.99990,...,157.60013,23.0,3.514600,2756.194500,9554.031500,FI1D9,19.109306,crop_yield,"""+5""",Inhibitors (IS) (>6.5 ph)
33589,16480,26.538750,64.976831,439.90015,562.50010,543.30005,412.29996,438.40002,631.80020,609.50010,...,144.66957,23.0,3.514600,2756.194500,9554.031500,FI1D9,5.213287,crop_yield,"""+5""",Inhibitors (IS) (>6.5 ph)
33590,16481,26.856371,64.976831,460.60016,593.30010,565.60004,418.50000,429.00015,655.39980,637.10000,...,147.73929,23.0,3.514600,2756.194500,9554.031500,FI1D9,11.664574,crop_yield,"""+5""",Inhibitors (IS) (>6.5 ph)
33591,16482,27.173992,64.976831,472.00012,602.69990,581.40000,425.00006,428.80014,668.99990,653.30005,...,147.46213,23.0,3.514600,2756.194500,9554.031500,FI1D9,26.302369,crop_yield,"""+5""",Inhibitors (IS) (>6.5 ph)


### Saving the raw data after combining

In [29]:
combined_fin.to_csv('data_raw.csv', index=False)

### Processing one of the collumns to revieve the yield effect as a float, this will be used later to select the best farming practice per grid

In [30]:
test = pd.DataFrame()
test['results_actual'] = combined_fin["results_actual"].str.split(",")
for idx, row in test.iterrows():
    test.iloc[idx]['results_actual'] = test.iloc[idx]['results_actual'][0].strip('"\'')
test['results_actual'] = test['results_actual'].astype(float)
combined_fin['yield_effect'] = test['results_actual']

C:\Users\artur\AppData\Local\Temp\ipykernel_18968\2399188711.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test.iloc[idx]['results_actual'] = test.iloc[idx]['results_actual'][0].strip('"\'')


### Filtering for such that only the best farming practice is left per grid

In [31]:
data_with_yield_dist_only_best = combined_fin.loc[combined_fin.groupby(['lon', 'lat'])['yield_effect'].idxmax()]
data_with_yield_dist_only_best

,Unnamed: 0,lon,lat,yearly_rain_6,yearly_rain_7,yearly_rain_8,yearly_rain_9,yearly_rain_10,yearly_rain_11,yearly_rain_12,...,MostProbable,yield,harvest_area,production,adm_id,nearest_distance_km,results_detailed,results_actual,farming_practice,yield_effect
9720,3282,-9.352423,38.614288,790.40010,359.20007,524.10010,638.20010,840.99980,578.10010,438.99997,...,18.0,2.535143,1343.657143,3370.342857,PT17,30.112339,"yield,wue","""+25,+28""",aerated irrigation (AI) (ph <7),25.00
9739,3399,-9.352423,38.931909,790.70013,365.00000,533.60020,667.60010,840.90010,591.50010,428.00003,...,18.0,2.535143,1343.657143,3370.342857,PT17,34.316214,"yield,wue","""+25,+28""",aerated irrigation (AI) (ph <7),25.00
19741,3283,-9.034802,38.614288,714.20010,338.20010,471.90010,560.00000,806.09990,551.89996,420.80000,...,18.0,2.535143,1343.657143,3370.342857,PT17,13.802448,yield,"""+49.1""",Film-mulching drip irrigation (Spain),49.10
19755,3400,-9.034802,38.931909,741.39984,358.70000,494.50015,625.30000,821.20013,584.00000,427.19998,...,18.0,2.535143,1343.657143,3370.342857,PT17,21.553158,yield,"""+49.1""",Film-mulching drip irrigation (Spain),49.10
19742,3284,-8.717181,38.614288,652.10000,338.30005,440.39996,515.50010,769.90020,534.79987,388.40000,...,18.0,2.535143,1343.657143,3370.342857,PT17,31.539169,yield,"""+49.1""",Film-mulching drip irrigation (Spain),49.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18702,15570,29.714960,63.071105,535.60016,623.59985,630.70010,437.90002,428.90020,595.00006,655.80000,...,23.0,3.001750,1462.500000,4437.500000,FI1D3,20.446827,yield,"""+26.04""",Soil Mulching (<13 Celcious),26.04
18688,15408,30.032581,62.753484,536.90020,634.89970,643.10020,458.40018,447.40020,588.49980,659.49976,...,23.0,3.001750,1462.500000,4437.500000,FI1D3,18.906646,yield,"""+26.04""",Soil Mulching (<13 Celcious),26.04
18703,15571,30.032581,63.071105,547.40000,633.79990,639.59990,453.70020,434.40010,588.90027,667.09973,...,23.0,3.001750,1462.500000,4437.500000,FI1D3,17.642467,yield,"""+26.04""",Soil Mulching (<13 Celcious),26.04
18689,15409,30.350202,62.753484,557.30010,638.69980,652.99994,473.50006,454.00030,585.90010,674.19990,...,23.0,3.001750,1462.500000,4437.500000,FI1D3,27.738901,yield,"""+26.04""",Soil Mulching (<13 Celcious),26.04


### Saving the data that has been filtered for the best farming practice per grid

In [32]:
data_with_yield_dist_only_best.to_csv('data_distanceFilter_best.csv', index=False)